# Program for incorporating OpenFoodFacts into the project
This program describes how the JSON has been achieved by collecting the nutritional information of the ingredients of each dish, averaging and incorporating them into the JSON file that collects the information about each meal.

Originally, the JSON had been generated by ChatGPT. This program is responsible for adding the nutritional information of the ingredients of each dish using the OpenFoodFacts database.

Implementation by: Javier Maciá Sempere (https://github.com/Javier-Macia)

Check the source of the code: https://world.openfoodfacts.org/data

In [ ]:
# We import the necessary libraries.
import pandas as pd
import dask.dataframe as dd
import json

In [ ]:
# We define the expected data types in the columns to avoid conflicts when operating on the data.
dtypes = {
    'abbreviated_product_name': 'object',
    'additives_en': 'object',
    'additives_tags': 'object',
    'allergens': 'object',
    'brand_owner': 'object',
    'cities_tags': 'object',
    'code': 'object',
    'emb_codes': 'object',
    'emb_codes_tags': 'object',
    'first_packaging_code_geo': 'object',
    'generic_name': 'object',
    'last_updated_t': 'float64',
    'manufacturing_places': 'object',
    'manufacturing_places_tags': 'object',
    'no_nutrition_data': 'object',
    'owner': 'object',
    'packaging': 'object',
    'packaging_en': 'object',
    'packaging_tags': 'object',
    'packaging_text': 'object',
    'purchase_places': 'object',
    'serving_size': 'object',
    'stores': 'object',
    'traces': 'object',
    'traces_en': 'object',
    'traces_tags': 'object',
    'additives_n': 'object',
    'cities': 'object',
    'nutriscore_score': 'object',
    'serving_quantity': 'object'
}


In [ ]:
# We read the CSV file. Openfoodfacts.data.csv is an 8gb file and, for that reason,
# we use dask to read it. The delimiter is a tab.
# ---------------IMPORTANT----------------
# Because the file is very large, it has not been uploaded to Github. Therefore, it must be downloaded and placed in the data folder.
ddf = dd.read_csv('./data/openfoodfacts.data.csv', delimiter='\t' ,dtype=dtypes)

In [ ]:
# We load the JSON with the old food information 
with open('./data/old_food_info.json') as f:
    food_info = json.load(f)

In [ ]:
def get_average_nutrition(ddf: dd.DataFrame , ingredient: str) -> pd.Series:
    """Function to get the average nutritional values of an ingredient in the Dask DataFrame

    Args:
        ddf (dd.DataFrame): Dask DataFrame with nutritional information of foods
        ingredient (str): Name of the ingredient to search for

    Returns:
        pd.Series: Average nutritional values of the ingredient
    """
    # We filter the Dask DataFrame by the ingredient name.
    # This way, we will only be left with the rows that contain the ingredient in their name.
    filtered = ddf[ddf['generic_name'].str.contains(ingredient, case=False, na=False)]
    
    # Select nutritional columns
    nutrition_columns = ['energy_100g', 'sugars_100g', 'fiber_100g', 'fat_100g', 'proteins_100g', 'carbohydrates_100g']
    
    # Calculate the average values
    average_nutrition = filtered[nutrition_columns].mean().compute()
    
    return average_nutrition

def get_recipe_nutrition(ddf: dd.DataFrame, recipe: dict) -> pd.Series:
    """Function to obtain the nutritional values of a recipe

    Args:
        ddf (dd.DataFrame): Dask DataFrame with nutritional information of foods
        recipe (dict): Recipe with ingredients and portion
        
    Returns:
        pd.Series: Nutritional values of the recipe
    """
    ingredients = recipe['ingredients']
    portion_size = recipe['portion']
    
    total_nutrition = pd.Series(0, index=['energy_100g', 'sugars_100g', 'fiber_100g', 'fat_100g', 'proteins_100g', 'carbohydrates_100g'])
    count = 0
    
    # For each ingredient in the dish, we calculate the average nutritional values
    for ingredient in ingredients:
        average_nutrition = get_average_nutrition(ddf, ingredient)
        # If the ingredient has no nutritional values, we do not count it
        if not average_nutrition.isnull().any():
            total_nutrition += average_nutrition
            count += 1
    
    # If there is more than one ingredient, we calculate the average nutritional values
    if count > 0:
        average_nutrition = total_nutrition / count
    else:
        average_nutrition = total_nutrition
    
    # Adjust nutritional values ​​according to serving
    multiplier = portion_size / 100
    recipe_nutrition = average_nutrition * multiplier
    
    return recipe_nutrition

In [ ]:
# Calculate nutritional values ​​for each recipe in the JSON
for recipe_name, recipe_info in food_info.items():
    nutrition = get_recipe_nutrition(ddf, recipe_info)
    food_info[recipe_name]['nutrition'] = nutrition.to_dict()

# Save the updated JSON
with open('./data/calculated_food_info.json', 'w') as f:
    json.dump(food_info, f, indent=4)

In [ ]:
# The previous operation may have taken a long time. We may have logged out once the calculations.
# Run this cell if you want to load the JSON with the OpenFoodFacts food information.
food_info = json.load(open('./data/calculated_food_info.json', 'r'))

In [ ]:
# We look for those cases in which the values ​​of 'energy_100g',
# 'sugars_100g', 'fiber_100g', 'fat_100g', 'proteins_100g' and 'carbohydrates_100g'
# be, all, 0. If so, the values ​​that had been obtained from ChatGPT will be taken
# and will be replaced by these.
for recipe_name, recipe_info in food_info.items():
    if recipe_info['nutrition']['energy_100g'] == 0 and recipe_info['nutrition']['sugars_100g'] == 0 and recipe_info['nutrition']['fiber_100g'] == 0 and recipe_info['nutrition']['fat_100g'] == 0 and recipe_info['nutrition']['proteins_100g'] == 0 and recipe_info['nutrition']['carbohydrates_100g'] == 0:
        print(recipe_name, recipe_info['nutrition'])
        recipe_info['nutrition']['sugars_100g'] = recipe_info['sugars']
        recipe_info['nutrition']['fiber_100g'] = recipe_info['fiber']
        recipe_info['nutrition']['fat_100g'] = recipe_info['fat']
        recipe_info['nutrition']['proteins_100g'] = recipe_info['protein']
        recipe_info['nutrition']['carbohydrates_100g'] = recipe_info['carbohydrates']
        # The energy had been obtained from ChatGPT, but can be calculated with the following operation to give more precise information: kcal = 9 x fats + 4 x carbohydrates + 4 x proteins
        recipe_info['nutrition']['energy_100g'] = recipe_info['nutrition']['fat_100g']*9 + recipe_info['nutrition']['carbohydrates_100g']*4 + recipe_info['nutrition']['proteins_100g']*4
        print(recipe_name, recipe_info['nutrition'])

In [ ]:
# We take into account what is generated by ChatGPT to calculate nutritional values
# We do this in each of the dishes and we do it in order to try to have more precise information.
# Energy is not taken into account since ChatGPT has given data that does not fit the aforementioned formula (kcal = 9 x fats + 4 x carbohydrates + 4 x proteins).
# For this reason, its value is recalculated.
for recipe_name, recipe_info in food_info.items():
    recipe_info['sugars'] = (recipe_info['nutrition']['sugars_100g'] + recipe_info['sugars'])/2
    recipe_info['fiber'] = (recipe_info['nutrition']['fiber_100g'] + recipe_info['fiber'])/2
    recipe_info['fat'] = (recipe_info['nutrition']['fat_100g'] + recipe_info['fat'])/2
    recipe_info['protein'] = (recipe_info['nutrition']['proteins_100g'] + recipe_info['protein'])/2
    recipe_info['carbohydrates'] = (recipe_info['nutrition']['carbohydrates_100g'] + recipe_info['carbohydrates'])/2
    recipe_info['kcal'] = recipe_info['fat']*9 + recipe_info['carbohydrates']*4 + recipe_info['protein']*4

In [ ]:
# We round nutritional values ​​so that there are no decimals
for recipe_name, recipe_info in food_info.items():
    recipe_info['sugars'] = round(recipe_info['sugars'])
    recipe_info['fiber'] = round(recipe_info['fiber'])
    recipe_info['fat'] = round(recipe_info['fat'])
    recipe_info['protein'] = round(recipe_info['protein'])
    recipe_info['carbohydrates'] = round(recipe_info['carbohydrates'])
    recipe_info['kcal'] = round(recipe_info['kcal'])

In [ ]:
# We remove all "nutrition" from food information
for recipe_name, recipe_info in food_info.items():
    del recipe_info['nutrition']

In [ ]:
# Optional printing JSON with OpenFoodFacts food information.
# We do this to validate that everything has been done correctly.
print(json.dumps(food_info, indent=4))

In [ ]:
# We save the JSON with the food information from OpenFoodFacts
with open('./data/food_info.json', 'w') as f:
    json.dump(food_info, f, indent=4)